In [10]:
import numpy as np
import qutip
import pickle

from optimparallel import minimize_parallel

from CoupledQuantumSystems.drive import DriveTerm, square_pulse_with_rise_fall
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process
from concurrent.futures import ProcessPoolExecutor
# -------------------------------
# Define your parameters and objects
# -------------------------------
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [
    qutip.basis(qbt.truncated_dim, i) * qutip.basis(qbt.truncated_dim, i).dag()
    for i in range(10)
]

element = np.abs(
    qbt.fluxonium.matrixelement_table("n_operator", evals_count=3)[1, 2]
)
freq = (qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]) * 2 * np.pi

# -------------------------------
# Objective function
# -------------------------------
def objective(t_tot, amp, w_d,ramp):
    tlist = np.linspace(0, t_tot * (1+ramp), 100)

    initial_states = [
        qutip.basis(qbt.truncated_dim, 1),
        qutip.basis(qbt.truncated_dim, 2),
    ]

    drive_terms = [
        DriveTerm(
            driven_op=qutip.Qobj(qbt.fluxonium.n_operator(energy_esys=True)),
            pulse_shape_func=square_pulse_with_rise_fall,
            pulse_id="pi",
            pulse_shape_args={
                "w_d": w_d,    # No extra 2pi factor
                "amp": amp,    # No extra 2pi factor
                "t_square": t_tot * (1-ramp),
                "t_rise": t_tot * ramp,
            },
        )
    ]

    # Solve the dynamics for each initial state
    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
            print_progress=False,
        )
        results.append(res)

    # Calculate final populations
    # (We want to transfer |1> -> |2> and |2> -> |1>.)
    one_minus_pop2 = abs(1 - results[0].expect[2][-1])  # from state |1> to |2>
    one_minus_pop1 = abs(1 - results[1].expect[1][-1])  # from state |2> to |1>

    result =  one_minus_pop2 + one_minus_pop1
    print(result)
    return result

# ---------------------------------------------------------------------
# SET UP NEVERGRAD INSTRUMENTATION
# ---------------------------------------------------------------------
# We'll specify a search space for each of t_tot, amp, w_d.
# For example, let's guess t_tot is in [1, 300].
# We'll assume amp in [0, 0.1], w_d in [freq*0.9, freq*1.1] as an example.
# Adjust these bounds to your scenario.


t_tot_list = np.linspace(50, 250, 21)
for t_tot in t_tot_list:
    # init_amp = np.pi / element / t_tot / 3.1415 / 2
    init_wd = qbt.fluxonium.eigenvals()[2] - qbt.fluxonium.eigenvals()[1]
    amp_guess = 50/t_tot * 22

    def fixed_t_tot_objective(x):
        amp, w_d,ramp = x
        return objective(t_tot=t_tot, amp=amp, w_d=w_d,ramp=ramp)
    
    result = minimize_parallel(fun=fixed_t_tot_objective,
                       x0=[amp_guess, init_wd, 0.15], 
                       bounds=((amp_guess/5, amp_guess*5), (0.003, 0.02), (1e-10, 0.5)),
                    # bounds=((0, np.inf), (0.003, 0.02), (1e-10, 0.5)),
                       parallel={'loginfo': True, 'time': True})

    best_cost = result['fun']
    best_amp = result['x'][0]
    best_w_d = result['x'][1]
    best_ramp = result['x'][2]

    print(f"Best cost found: {best_cost}")
    print(f"Best amp: {best_amp}, best w_d: {best_w_d}, best ramp: {best_ramp}")

    results_dict = {
        "best_cost": best_cost,
        "best_amp": best_amp,
        "best_w_d": best_w_d,
        "best_ramp": best_ramp,
        "result": result,
    }

    with open(f"optimparallel_optimized_results_{t_tot}.pkl", "wb") as f:
        pickle.dump(results_dict, f)


1.574415681745677
1.5755296469795461
1.5744408118563977
1.5744101406266418
1.9202920000817811
1.9190823856818842
1.9203000124388008
1.9202663975063867
1.2016658929070858
1.1966396259011627
1.196636921646286
1.1964676058889396
0.38563191734165314
0.38576114825655705
0.38561656295494173
0.3871100088099789
0.22624340887865724
0.22631617171653273
0.2262412001993077
0.22623423447128
1.8561187002293642
1.8561287178891304
1.8560961397230291
1.8567474356480476
1.6469302926854463
1.648194735504764
1.6481455740419526
1.6481704780863078
1.2351556468175369
1.2351551186704794
1.2352459609325492
1.2364005611956859
0.5760192422890689
0.5775704998839271
0.5760163177902246
0.576046353730542
0.8433315333326623
0.8433149357035234
0.843335626198165
0.8429507148907088
0.7376894467213727
0.7383642705513612
0.7376878247496085
0.7377078912558777
0.9790413125506123
0.979646219404741
0.9796330805093185
0.979634400775862
1.7303625155102618
1.7303507885616058
1.730318323381326
1.7295612906106268
0.333880961999182

KeyError: 'fval'